In [ ]:
!pip install transformers datasets torch
!pip install accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset

# Load GoEmotions dataset
dataset = load_dataset("go_emotions")

# Check dataset structure
print(dataset)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.40k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.77M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/350k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/347k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/43410 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5426 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5427 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 43410
    })
    validation: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 5426
    })
    test: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 5427
    })
})


In [ ]:
from transformers import AutoTokenizer

# Load pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Function to tokenize the text
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

# Tokenize dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

In [ ]:
import torch

# Convert dataset into PyTorch format
tokenized_datasets.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# Prepare train & test datasets
train_dataset = tokenized_datasets["train"]
val_dataset = tokenized_datasets["validation"]
test_dataset = tokenized_datasets["test"]


In [ ]:
from transformers import AutoModelForSequenceClassification

# Load pre-trained DistilBERT model with 27 output labels (GoEmotions has 27 emotions)
num_labels = 27
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_labels)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments, Trainer

# Training settings
training_args = TrainingArguments(
    output_dir="./go_emotions_results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
def preprocess_function(examples):
    return tokenizer(
        examples["text"],
        padding=True,
        truncation=True,
        max_length=128,
    )

tokenized_datasets = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

In [ ]:
import torch

def convert_labels_to_tensor(examples):
    labels = examples["labels"]


    if isinstance(labels[0], float):

        multi_hot = torch.zeros((len(labels), num_labels), dtype=torch.float32)
        for i, sample_labels in enumerate(labels):
            if 0 <= int(sample_labels) < num_labels:
                multi_hot[i, int(sample_labels)] = 1
        examples["labels"] = multi_hot
    else:

        max_labels = max(len(sample_labels) for sample_labels in labels)
        padded_labels = [sample_labels + [-1] * (max_labels - len(sample_labels)) for sample_labels in labels]

        multi_hot = torch.zeros((len(labels), num_labels), dtype=torch.float32)
        for i, sample_labels in enumerate(padded_labels):
            for label in sample_labels:
                if 0 <= label < num_labels:
                    multi_hot[i, label] = 1

        examples["labels"] = multi_hot

    return examples

tokenized_datasets = tokenized_datasets.map(convert_labels_to_tensor, batched=True)

Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    args=training_args,
)
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.068500,0.065668


Epoch,Training Loss,Validation Loss
1,0.068500,0.065668
2,0.062100,0.061209
3,0.054900,0.061296
4,0.046000,0.062801
5,0.040800,0.063769


TrainOutput(global_step=13570, training_loss=0.05970517953881114, metrics={'train_runtime': 1999.5861, 'train_samples_per_second': 108.547, 'train_steps_per_second': 6.786, 'total_flos': 5358247104575196.0, 'train_loss': 0.05970517953881114, 'epoch': 5.0})

In [ ]:
# Save trained model & tokenizer
model.save_pretrained("go_emotions_model")
tokenizer.save_pretrained("go_emotions_model")

# Zip for download
!zip -r go_emotions_model.zip go_emotions_model


  adding: go_emotions_model/ (stored 0%)
  adding: go_emotions_model/config.json (deflated 64%)
  adding: go_emotions_model/model.safetensors (deflated 8%)
  adding: go_emotions_model/tokenizer_config.json (deflated 75%)
  adding: go_emotions_model/vocab.txt (deflated 53%)
  adding: go_emotions_model/special_tokens_map.json (deflated 42%)
  adding: go_emotions_model/tokenizer.json (deflated 71%)


In [ ]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
import torch

#Testing the fine tuned model
model_path = "go_emotions_model"
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)


emotion_classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, top_k=None)


emotions = emotion_classifier("What a bright sunny day!")


most_confident = max(emotions[0], key=lambda x: x['score'])


label_mapping = {
    0: "Admiration", 1: "Amusement", 2: "Anger", 3: "Annoyance", 4: "Approval",
    5: "Caring", 6: "Confusion", 7: "Curiosity", 8: "Desire", 9: "Disappointment",
    10: "Disapproval", 11: "Disgust", 12: "Embarrassment", 13: "Excitement",
    14: "Fear", 15: "Gratitude", 16: "Grief", 17: "Joy", 18: "Love",
    19: "Nervousness", 20: "Optimism", 21: "Pride", 22: "Realization",
    23: "Relief", 24: "Remorse", 25: "Sadness", 26: "Surprise"
}

# Convert numerical label to emotion name
emotion_label = label_mapping[int(most_confident['label'].split('_')[-1])]

# Print result
print(f"Detected emotion: {emotion_label} (Confidence: {most_confident['score']:.4f})")


Device set to use cuda:0


Detected emotion: Excitement (Confidence: 0.5366)
